In [7]:
%matplotlib inline
%load_ext autoreload

import sys
import csv
import multiprocessing

import ujson
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
import tensorflow as tf
import gensim
from gensim.models.fasttext import FastText
from gensim.parsing.preprocessing import preprocess_string
import pandas as pd
from tqdm import tqdm 
import seaborn as sns
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
csv.field_size_limit(500 * 1024 * 1024)

524288000

In [9]:
path_data = 'data/fake_news_corpus/'
path_news_csv = path_data + 'news_cleaned_2018_02_13.csv'
path_fasttext = path_data + 'news_cleaned_2018_02_13.fasttext.bin'
path_news_preprocessed = path_data + 'news_cleaned_2018_02_13.preprocessed.jsonl'
path_news_shuffled = path_data + 'news_cleaned_2018_02_13.preprocessed.shuffled.jsonl'
path_news_train = path_data + 'news_cleaned_2018_02_13.preprocessed.shuffled.train.jsonl'
path_news_test = path_data + 'news_cleaned_2018_02_13.preprocessed.shuffled.test.jsonl'
path_news_val = path_data + 'news_cleaned_2018_02_13.preprocessed.shuffled.val.jsonl'

path_news_embedded = path_data + 'news_cleaned_2018_02_13.embedded.jsonl'

# Load FastText

In [ ]:
fasttext = FastText.load_fasttext_format(path_fasttext)

# Load news_cleaned

In [ ]:
for df_news_chunk in pd.read_csv(path_news_csv, chunksize=1000):
    break

# Prepare the input data 

## Make sure the file is correct

In [ ]:
with tqdm() as progress:
    for df_news_chunk in pd.read_csv(path_news_csv, encoding='utf-8', engine='python', chunksize=1000):
        progress.update()

## News generator

In [ ]:
def news_generator():
    with tqdm() as progress:
        for df_news_chunk in pd.read_csv(path_news_csv, encoding='utf-8', engine='python', chunksize=10 * 1000):
            news_filter = df_news_chunk.type.isin(set(['fake', 'conspiracy', 'unreliable', 'reliable']))
            df_news_chunk_filtered = df_news_chunk[news_filter]
            for row in df_news_chunk_filtered.itertuples():
                label = 1 if row.type == 'reliable' else 0

                progress.update()
                yield int(row.id), '%s %s' % (row.title, row.content), label

## Show text length distribution

In [ ]:
lens = []
for i, (_id, con, label) in enumerate(news_generator()):
    if i > 10 * 1000:
        break

    lens.append(len(con))
    progress.update()

In [ ]:
len(lens), sum(lens) / len(lens), max(lens), min(lens)

In [5]:
sns.distplot(lens)

NameError: name 'lens' is not defined

## Preprocess  

In [ ]:
def _preprocess_string(news):
    _id, con, label = news
    return _id, preprocess_string(con), label

def news_preprocessed_generator():
    missing_words = {}
    
    with multiprocessing.Pool(multiprocessing.cpu_count(), maxtasksperchild=1) as pool:
        for _id, con, label in pool.imap(_preprocess_string, news_generator(), chunksize=1000):
            yield _id, con, label, missing_words

In [ ]:
all_missing_words = {}
with open(path_news_preprocessed, 'w') as out_news_embedded:
    for _id, con, label, missing_words in news_preprocessed_generator():
        out_news_embedded.write(ujson.dumps({
            'id': _id, 'content': con, 'label': int(label)
        }) + '\n')
        all_missing_words.update(missing_words)

In [ ]:
!shuf data/fake_news_corpus/news_cleaned_2018_02_13.preprocessed.jsonl > \
      data/fake_news_corpus/news_cleaned_2018_02_13.embedded.preprocsessed.jsonl

## Split into train, validation, and test

In [ ]:
count_lines = 0
with open(path_news_shuffled, 'r') as in_news:
    for line in in_news:
        count_lines += 1

In [ ]:
count_lines, int(count_lines * .8), int(count_lines * .1), count_lines - (int(count_lines * 0.8) + int(count_lines * 0.1))

In [ ]:
train_size = int(count_lines * .8)
test_size = int(count_lines * .8)
val_size = count_lines - (int(count_lines * 0.8) + int(count_lines * 0.1))

In [ ]:
with open(path_news_shuffled, 'r') as in_news:
    with open(path_news_train, 'w') as out_train:
        with open(path_news_test, 'w') as out_test:
            with open(path_news_val, 'w') as out_val:
                for i, line in tqdm(enumerate(in_news)):
                    if i < count_lines * .8:
                        out_train.write(line)
                    elif i < count_lines * .9:
                        out_test.write(line)
                    else:
                        out_val.write(line)

## Generators

In [ ]:
max_words = 1000

In [ ]:
def _news_generator_process_line(line):

    article = ujson.loads(line)

    embedding = np.zeros((max_words, 100))
    for i, word in enumerate(article['content'][:300]):
        if word in fasttext:
            embedding[i] = fasttext[word]
            
    return embedding, article['label']

def news_generator(path, batch):
    while True:
        with open(path_news_shuffled, 'r') as in_news:
            with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
                batch_i = 0
                batch_embedding = np.zeros((batch, max_words, 100))
                batch_label = np.zeros((batch, 1))
                for embedding, label in pool.imap(_news_generator_process_line, in_news, chunksize=250):
                    if (batch_i + 1) == batch:
                        yield batch_embedding, batch_label
                        batch_embedding = np.zeros((batch, max_words, 100))
                        batch_label = np.zeros((batch, 1))
                        batch_i = 0
                    else:
                        batch_embedding[batch_i] = embedding
                        batch_label[batch_i, 0] = label
                        batch_i += 1

# Simple CNN model for classyfing fake vs reliable

In [ ]:
input_shape = max_words, 100

filters = 250
kernel_size = 3
hidden_dims = 250

batch_size = 32
epochs = 3

In [ ]:
def cnn_simple_model(input_shape, filters=250, kernel_size=3, hidden_dims=250):
    model = Sequential()
    model.add(Conv1D(filters, kernel_size, input_shape=(input_shape[0], input_shape[1]), padding='valid', 
               activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
#     model.add(Conv1D(filters, kernel_size, activation='relu'))
#     model.add(GlobalMaxPooling1D())
    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
with tf.device('/gpu:0'):
    cnn_model = cnn_simple_model(input_shape, filters, kernel_size, hidden_dims)
    cnn_model.fit_generator(news_generator(path_news_train, batch_size), steps_per_epoch=train_size // batch_size, 
                            epochs=epochs, verbose=1, validation_data=news_generator(path_news_val, batch_size), 
                            validation_steps=val_size // batch_size)